# MS141 Lecture 11

# Introduction to Ordinary Differential Equations (ODEs)

## Read: Newman's book, first part of Chapter 8 (pages 327 - 342).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

%matplotlib inline

# Set common figure parameters
newparams = {'figure.figsize': (10, 6), 'axes.grid': True,
             'lines.linewidth': 1.5, 'lines.markersize': 10,
             'font.size': 16}
plt.rcParams.update(newparams)

## 1. Brief intro to ODEs

A [differential equation](https://en.wikipedia.org/wiki/Differential_equation) relates a function $y$ with its derivatives. The function usually represents a physical quantity and the derivatives its rate of change; the differential equations relates them. Because such relations are extremely common, differential equations play a prominent role in science and engineering.

Differential equations can be classified into several types, including ordinary and partial, linear and non-linear, and homogeneous or inhomogeneous.<br> 
**Ordinary differential equations (ODEs)** contain an unknown function of one real or complex variable $x$, its derivatives, and some given function of $x$.<br> 
The unknown function is generally represented by a variable (often denoted as $y$), which depends on $x$, the independent variable in the equation.<br> Linear differential equations are linear in the unknown function and its derivatives $-$ namely, these quantities appear only with a power of 1. Most ODEs encountered in science are linear, but nonlinear equations are also important.<br>

The term "ordinary" is used to differentiate ODEs from **partial differential equations (PDEs)**, namely equations in which the unknown function depends on more than one independent variable and the partial derivatives with respect to these variables.

Differential equations are described by their **order, which is determined by the highest derivative**. An equation containing only first derivatives is a first-order differential equation, an equation containing a second derivative is a second-order differential equation, and so on. The solution to a first order ODE always contains one arbitrary constant, the solution to a second order ODE two such constants, etc. For any specific problem, the value of the constants is fixed by the **boundary conditions**. 

For ODEs, when the independent variable has the meaning of time (e.g., in Newton's equation), we call such conditions **initial conditions**; they typically include the position and velocity at time zero, but the key is that all the conditions are specified at the same point. This type of ODE is called **initial value problem**.

When the independent variable has the meaning of space, the boundary conditions typically fix the value of the unknown function and/or its derivatives at the boundaries of the solution domain, and thus typically at multiple points. This type of ODE is called a **boundary value problem**. 

For PDEs, in which both time and space variables can appear, one can have both initial and boundary values. An example of a PDE is the time-dependent Schrodinger's equation of quantum mechanics, in which the first time derivative appears together with the space variable and its derivatives.

## 2. Numerical methods for ODEs

Many differential equations cannot be solved analytically. [Numerical methods for differential equations](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations) can be used to find approximate solutions with a given degree of accuracy. We will explore basic methods to solve first and second order ODEs in this and the next lecture. The goal is not to be comprehensive, but rather, to focus on a few widely applicable and robust methods and outline key concepts.


## 2.1 Euler's method
The simplest numerical approach to solve ODEs is [Euler's method](https://en.wikipedia.org/wiki/Euler_method). It is simple to understand and implement, but not very accurate or stable.<br> 
We want to solve the **first order ODE** with general form: 

$$\frac{dy}{dt} = f( t , y)$$ 

where $f(t,y)$ is typically called the *derivative function*. Here, we'll interpret the independent variable as time, and thus we'll need to specify one initial condition for the value of the function at time zero, $y(t_0) = y_0$. The first order ODE plus its initial condition are a well-posed problem that can be solved numerically. When solving this equation numerically, we look for a solution $y(t)$ over a certain time domain ranging from $t=0$ to a final time. 

The Euler algorithm can be summarized in four points:
1. We first discretize time, defining a regular time grid $t_n$ with a constant time step $h = t_{n+1} - t_n$:

$$t_n = t_0 + n \cdot h~~~~~\mathrm{with}~~~~n=0,\,1,\,2,\,3,\,...,\,N$$

2. Euler's method approximates the derivative using the forward difference formula, and evaluates the derivative function at the **current** step:

$$ \frac{ y(t_{n+1}) - y(t_{n}) }{ h } \,\approx\, f(\,t_n,\,y(t_n)\,) $$

3. Defining a more compact notation in which $y(t_n) = y_n$, the Euler method advances the function $y$ to the next step using:

$$\boxed{ y_{n+1} = y_n + f(y_n,t_n)\,h }$$

4. Starting from the initial condition $y(t_0) = y_0$, the unknown function $y(t)$ is time-stepped for a total of $N$ steps by computing $y_{n+1}$ from the value of $y_n$.

We implement the Euler method and test it on a simple ODE:

$$ \frac{dy}{dt} = \,2t\, (1 + y^2)\,~~~~~, ~~~~~y(0) = 0 $$

which has the exact solution $y(t) = \tan(t^2)$.

In [ ]:
# Simple Euler implementation

t=0; y=0; # initial condition
h=0.01; N=50 # time grid

for i in range(N):
    
    y += 2*t*(1 + y**2)*h #advance y using f(t,y)
    t = t + h # advance time
    print ("%.2f " % t, " %.6f" % y);

We can compare the approximate Euler solution at $t = 0.5$, $y_{E}(0.5) = 0.244244 $ with the exact solution, 
 $y(0.5)$ = 0.255342.<br> 
 The total error after 25 time steps is $E = 0.0110978$, and thus as large as roughly 4$\%$.
 
We now try using a smaller value for the time step, $h = 0.01$, which is half of the value we used previously. We obtain an approximate Euler solution at $t = 0.5$ of $y_E (0.5) = 0.249770 $. The error is now $E = 0.005571$, which is half of the error with the twice larger time step. 

This simple numerical experiment suggests that the global error (after $N$ time steps) 
is proportional to $h$ in the Euler method.<br> 
We say that the Euler method is first-order (that is, order $h$) accurate. A better accuracy would be desirable, since with first-order accuracy one needs to use a small step size $h$ to obtain sensible results, thus increasing computational cost and rounding error.

A better version of the Euler code is given below. This version stores the time and $y$ values in memory using `numpy` arrays,<br> 
making it easier to plot the solution after the calculation and compare the numerical and exact solutions.

In [ ]:
# function f(t,y)
def f(t,y):
    return (2*t*(1 + y**2))

# Euler method 
N = 25 # number of steps
h = 0.02  # time step

# initial values
t0 = 0.0
y0 = 0.0

t = np.zeros(N+1)
y_eu = np.zeros(N+1)
t[0] = t0
y_eu[0] = y0

t_old = t0
y_old = y0

for n in range(N):
    y_new = y_old + h*f(t_old,y_old)  # Euler's method
    
    # store the solution
    t[n+1] = t_old + h
    y_eu[n+1] = y_new
    
    # update t, y
    t_old = t_old + h
    y_old = y_new

print(r'y_N = %f' % y_old)

# Plot x(t)
plt.plot(t, y_eu, '-b', label='Euler')

y_exact = np.tan(t**2)
plt.plot (t,y_exact, '-r',label='Exact')

plt.ylabel(r'$y(t)$')
plt.xlabel(r'$t$')

plt.legend()
plt.grid()
plt.show()

Apart from its fairly poor accuracy, the main problem with Euler's method is that it can
be unstable, i.e. the numerical solution can start to deviate from the exact solution
in dramatic ways. Usually, when this happens the numerical solution grows large in
magnitude while the exact solution remains small. 

A popular example to demonstrate this feature is the ODE

$$\frac{dy}{dt}=- k\, y~~~\mathrm{with}~~~k>0,~~~~ \mathrm{and}~~~ y(0)=y_0$$

The exact solution is $y(t)=y_0\,e^{-kt}$, which satisfies the ODE and the initial condition. The Euler method will compute the solution as

$$ y_{n+1}=y_n + h\cdot(-k\,y_n) = (1\,-\,k\,h)\, y_n. $$

Clearly, if $h>1/k$ ($kh>1$), then $y_n$ will oscillate between negative and positive numbers. 
If in addition $h > 2/k$, the solution will also grow without bounds in magnitude as time increases, 
since $ |y_{n+1}| = |1 - kh|^n\, y(0)$. We know that this is incorrect as the exact solution decays exponentially to zero.

On the other hand, when $k\,h < 1$ the numerical solution approaches zero as time increases, correctly capturing the behavior of the exact solution. 

We conclude that in the Euler method we need to make sure that the step size is small enough to avoid such instabilities.<br> 
For completeness, we verify these facts with the Euler code below, where we take for simplicity $k = 1$ and $y_0 = 1$.

In [ ]:
# function f(y)
def f(y):
    return (-y)

# Euler method 
N = 100 # number of steps
h = 2.1  # time step (try making it >1 and >2)

# initial values
t0 = 0.0
y0 = 1.0

t = np.zeros(N+1)
y = np.zeros(N+1)
t[0] = t0
y[0] = y0

t_old = t0
y_old = y0

for n in range(N):
    y_new = y_old + h*f(y_old)  # Euler's method
    
    # store the solution
    t[n+1] = t_old + h
    y[n+1] = y_new
    
    # update t, y
    t_old = t_old + h
    y_old = y_new

# Plot x(t)
plt.plot(t, y, '-b', label='Euler')

y_exact = np.exp(-t)
plt.plot (t,y_exact, '-r',label='Exact')

plt.ylabel(r'$y(t)$')
plt.xlabel(r'$t$')

plt.legend()
plt.grid()
plt.show();

## 2.2 Solving the stability problem $-$ the implicit Euler method

A question that arises is what to use in the evaluation of the derivative function $f(t,y)$. Before we have actually made the step, we don't know where we are going to end up in $y$, so we can't easily decide where in $y$ to evaluate $f$. The easiest approach is to recognize that we already have the value of $y$ at the current time step, $y_n$, so we can use it to evaluate $f$. The Euler method uses this approach and evaluates $f$ as $f(t_n,y_n)$, using the value $y_n$ at the current time step.

<img src="images/Euler.png" style="width: 500px;"/>

Methods that use only data from the current (or previous) time steps are called **explicit** advancing schemes. 

Explicit methods are stable only for step size less than some value $-$ in the example above, the largest step size to avoid instabilities is $h < 2\,/\,k$. To overcome this limit, we can use **implicit** methods, in which the value of the derivative is computed at the *end* of the step rather than at the *beginning*. This leads to a stable method, at the cost of doing more intense calculations at each step. Implicit methods are particularly useful when the differential equation is *stiff*, namely it involves multiple timescales, some fast and some slow. In this case, with an implicit method we can use a longer time step without suffering from the instabilities associated with the faster time scale. 

We can define an implicit method called backward Euler (or implicit Euler), in which the advancing scheme is:

$$ \boxed{ y_{n+1} = y_n + f(t_{n+1}, y_{n+1}) h }$$

In general, this equation needs to be solved numerically using a root finding method, which leads to additional computations at each step. 

For our example above, namely the equation $y' = -k\,y$ for which $f(y_{n+1}) = - k\,y_{n+1}$, the solution is simple:

$$ y_{n+1} = y_n - k\,y_{n+1}\,h. $$

We rearrange this equation into 

$$ y_{n+1}( 1 + k\cdot h) = y_n $$

After $N$ steps, the numerical value of $y_n$ reads:

$$ y_n = (1+ k\cdot h)^{-n}\,y_0 $$

For positive $kh$ values (the case of interest here), this equation never becomes unstable, no matter how large $kh$ is, because the magnitude of the amplification factor $1/(1 + kh)$ is always smaller than 1. This is a characteristic of implicit schemes $-$ they are stable even for larger steps. 

We implement the backward Euler method below and apply it to our equation of interest.

In [ ]:
# implicit Euler for 
# y' = - y with y(0) = 1

N = 25 # number of steps
h = 2.01  # time step (test for h > 2.0)

# initial values
t0 = 0.0
y0 = 1.0

t = np.zeros(N+1)
y = np.zeros(N+1)
t[0] = t0
y[0] = y0

t_old = t0
y_old = y0

for n in range(N):
    # NOTE: THIS LINE IS VALID ONLY FOR y' = - y !
    # the amplification factor is known here
    y_new = y_old / (1. + h)
    
    # store the solution
    t[n+1] = t_old + h
    y[n+1] = y_new
    
    # update t, y
    t_old = t_old + h
    y_old = y_new

# Plot x(t)
plt.plot(t, y, '-b', label='Backward Euler')

y_exact = np.exp(-t)
plt.plot (t,y_exact, '-r',label='Exact')

plt.ylabel(r'$y(t)$')
plt.xlabel(r'$t$')

plt.legend()
plt.grid()
plt.show()

We can see that even for large time steps (in our case, $h > 2$), which made the Euler method unstable, backward Euler is still stable. Note that even though the method is stable, the accuracy is still order $h$ in backward Euler, therefore the error can be large. 

We also want to show how to apply backward Euler to a more general ODE, in which when we evaluate

$$ y_{n+1} = y_n + f(t_{n+1}, y_{n+1}) h$$

a numerical solution is needed. The problem is reduced to finding the zeros in the variable $y_{n+1}$, by solving

$$ y_{n+1} - y_n - f(t_{n+1}, y_{n+1}) h = 0$$

As we have studied in a previous lecture, we can solve this equation with the Newton method. Let's apply this approach to the ODE 

$$ \frac{dy}{dt} = 6y^2 \,t ~~~~~~,~~~~y(0)=1/28 $$

which has the exact solution $y(t) = \frac{1}{28 - 3t^2}$.

In [ ]:
# the function f and its derivative (for Newton)

def f(y, y_old, t_new): # y = y_n+1 is the independent variable in Newton
    return y - y_old - h * 6 * y**2 * t_new

def df(y, y_old, t_new):
    return 1.0 - 12.*h*y*t_new

N = 290 # number of steps
h = 0.01  # time step

# initial values
t0 = 0.0
y0 = 1./28.

t = np.zeros(N+1)
y = np.zeros(N+1)
t[0] = t0
y[0] = y0

t_old = t0
y_old = y0

for n in range(N):
    # these lines are valid in general
    t_new = t_old + h
    
    # use scipy's Newton implementation
    y[n+1] = optimize.newton(f, y_old, df, args=(y_old,t_new), maxiter=100)
    # debug
    # print (y[n+1])
    
    # store the solution
    t[n+1] = t_old + h
    
    # update t, y
    t_old = t_old + h
    y_old = y[n+1]

# Plot y(t)
plt.plot(t, y, '-b', label='Backward Euler')

y_exact = 1.0 / (28. - 3*t**2)
plt.plot (t,y_exact, '-r',label='Exact')

plt.ylabel(r'$y(t)$')
plt.xlabel(r'$t$')

plt.legend()
plt.grid()
plt.show();

This example showed how to add the Newton root finding step.<br> 
For this ODE, in which stability is not the main issue, Euler and backward Euler perform similarly, as you may verify.

## 2.3 Accuracy of the Euler method

The numerical evidence collected above for the Euler method suggests that its global accuracy after $N$ steps is of order $h$.<br> 
To show this result more rigorously, we first study the **local error** of the Euler method.

The local error is the error made in a single step. It is the difference between the numerical solution after one step, $y_1$, and the *exact* solution at time $t_1 = t_0\,+\,h$. The Euler numerical solution after one step is given by

$$ y_1^{(\rm{E})} = y_0 + f(t_0,y_0)\,h $$

The exact solution can be obtained by integrating the differential equation $ y' = f(t,y)$, which gives

$$  y_1 \,=\, y_0 \,+\, \int_{t_0}^{t_0 + h} f(t,y(t)) \,dt  $$

If we regard the function $f(t,y(t))$ as a function of $t$ only and computed at the exact solution $y(t)$, we can expand $f(t,y(t))$ in the integral as

$$ f(t,y(t)) = f(t_0,y_0) + \frac{df(t_0,y_0)}{dt} (t-t_0) + \frac{1}{2} \frac{d^2 f(t_0,y_0)}{dt^2}(t-t_0)^2 + \mathcal{O}((t-t_0)^3) $$

Substituting in the integral above and integrating, we get for the exact solution:

$$ y_1 \,=\, y_0 \,+\, f(t_0,y_0)\,h \,+\, \frac{df(t_0,y_0)}{dt} \frac{h^2}{2} \,+\, \mathcal{O}(h^3) $$

Finally, we can write the local error $e_{\mathrm{loc}}$ as the exact solution minus the Euler value:

$$ e_{\mathrm{loc}} = y_1 - y_1^{(\rm{E})} \,= \,\frac{df(t_0,y_0)}{dt} \frac{h^2}{2} + \ldots $$

We have thus shown that the local error is of order $h^2$, and thus the Euler method is first-order accurate. The global (cumulative) error can be estimated as the error accumulated after $N$ steps, $E = N \cdot e_{\mathrm{loc}}$. Since $N$ is the total time $t_f$ divided by the time step, we conclude that in Euler's method the global error is proportional to the step size $h$:

$$ E = e_{\mathrm{loc}} \frac{t_f}{h} \propto \frac{df(t_0,y_0)}{dt} \frac{h}{2}.$$

The same result applies to the implicit Euler method, which is also first-order accurate.

In principle, a higher accuracy for the numerical solution can be achieved by decreasing the step size $h$.
However, we cannot decrease $h$ indefinitely since ultimately round-off errors become dominant. Most importantly, using a smaller $h$ requires more steps for the same total time, and thus more computational work.   

## 2.4 More accurate methods for ODEs: Runge-Kutta

The error in Euler's method is due to the fact that we approximated the derivative function $f$ using its value only at the current step $t_n$. If however we evaluate $f(t,y)$ at more points before advancing to $t_{n+1}$ we can improve the accuracy of the method. For example, we can combine the value of $f$ at $t_n$ with its value at the next step $t_{n+1}$ or at the half step $t_{n+1/2} = t_n + \frac{h}{2}$, and use this information to more accurately advance to the next step and form the value of $y_{n+1}$. 

Methods using *multiple evaluations of the derivative function at each step for improving the accuracy are known as [Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods)*. They are explicit algorithms, so their stability should be tested for stiff equations, but they are robust and accurate. Below is a pictorial representation of their working principle.<br> 
(Image source: Wiki)
<img src="images/Runge-Kutta.png" style="width: 500px;"/>


### Runge-Kutta 2
For example, starting from the exact formula for $y_{n+1}$:

$$  y_{n+1} \,=\, y_n \,+\, \int_{t_n}^{t_n + h} f(t,y(t)) \,dt  $$

we can see that if we Taylor expand $f(t,y)$ about the half step $t_{n+1/2} = t_n + \frac{h}{2}$, we get

$$ f(t,y) = f(t_{n+1/2},y_{n+1/2}) + \frac{df(t_{n+1/2})}{dt}\,(t-t_{n+1/2}) + \mathcal{O}((t-t_{n+1/2})^2)$$

Plugging this expression in the equation for $y_{n+1}$ and integrating, we obtain

$$ \boxed{y_{n+1} = y_n + f(t_{n+1/2},y_{n+1/2})\,h + \mathcal{O}(h^3)}$$

since the integral of the term $(t-t_{n+1/2})$ vanishes. The error is now of order $h^3$ in the local step, and thus the cumulative error after $N$ steps is of order $h^2$.<br> 
This second order accurate method is known as Runge-Kutta 2 (where the 2 stands for second order accurate). The question is how to obtain the value $y_{n+1/2}$ at the half step. In Runge-Kutta, we **use the Euler method to obtain $y$ at the half step**, which does not lead to an error larger than $h^3$ in the formula above.

The Runge-Kutta 2 algorithm can be summarized as follows:
1. Evaluate $k_1 = h\,f(t_n,y_n)$
2. Use $k_1$ to obtain $f$ at the half step, forming $k_2 \,= \,h\, f(t_n + \frac{h}{2},y_n + \frac{k_1}{2})$
3. Advance the unknown function $y$ using the derivative at the half step: $y_{n+1} = y_n + k_2 $

All in all, we still use a regular grid $t_n = t_0 + n\,h$, advance one step at a time on this grid, and discard all information about previous steps. However, we now evaluate the function twice at each time step (with the help of the Euler method) using the advancing scheme:

$$ y_{n+1} = y_n + f(\,t + \frac{h}{2},y_n + \frac{h}{2}f_n \,)\,h + \mathcal{O}(h^3)$$

Runge-Kutta 2 is easy to implement (you are encouraged to try). 

### Runge-Kutta 4
A **widely used and robust algorithm for ODEs is Runge-Kutta 4 (or simply RK4)**. As the name suggests, RK4 is *4th order accurate*, meaning that the local truncation error is $\mathcal{O}(h^5)$ and the error after $N$ steps is $\mathcal{O}(h^4)$. RK4 uses *4 evaluations of the derivative function f(t,y)* at each step. For advancing to $y_{n+1}$, RK4 uses one evaluation of $f$ at the initial point $t_n$, two at the half step $t_{n} + \frac{h}{2}$, and one at the final point $t_{n+1}$ of the step. It can be shown that this approach is 4th order accurate using Simpson's rule; the derivation is a bit tedious, so we'll provide the final result.

The RK4 algorithm can be summarized as follows (see the figure above):
1. Evaluate $k_1 = h\,f(t_n,y_n)$
2. Similar to RK2, use $k_1$ to compute $k_2 \,= \,h\, f(t_n + \frac{h}{2},y_n + \frac{k_1}{2})$
3. Refine the half-step computation by obtaining $k_3 = \,h\, f(t_n + \frac{h}{2},y_n + \frac{k_2}{2})$
4. Compute the derivative function at $t_{n+1}$ through $k_4 = \,h\, f(t_n + h, y_n + k_3)$.

5. Advance the unknown function: $y_{n+1} = y_n + \frac{1}{6}\left( k_1 + 2\,k_2 + 2\,k_3 + k_4 \right)$

The related RK4 code is given below for the equation $ \frac{dy}{dt} = \,2t\, (1 + y^2)\,, ~~y(0) = 0 $ examined earlier with Euler.

In [ ]:
# 4th order Runge Kutta (adapted from NumFys)

# derivative function f (same as in Euler's method)
def f(y,t):
    return 2.0*t*(1. + y**2)

N = 25 # number of steps
h = 0.02  # time step

t = np.zeros(N+1)
y_4RK = np.zeros(N+1) 

# initial values
t[0] = 0.0
y_4RK[0] = 0.0

# 4 evaluations of f at each step (vs. just 1 for Euler)
for n in range(N):
    k1 = h*f( y_4RK[n]       , t[n]         )
    k2 = h*f( y_4RK[n] + k1/2, t[n] + (h/2) ) 
    k3 = h*f( y_4RK[n] + k2/2, t[n] + (h/2) ) 
    k4 = h*f( y_4RK[n] + k3  , t[n] +  h    )
    
    # advance to next step
    y_4RK[n+1] = y_4RK[n] + k1/6. + k2/3. + k3/3. + k4/6.
    t[n+1] = t[n] + h
    
print (y_4RK[N])

exact = np.tan(t**2)
plt.plot(t,exact, '-r', linewidth=3.0, label=r'exact')
plt.plot(t,y_4RK,'bo', linewidth=3.0, label=r'4th order Runge-Kutta')

# compare with Euler
plt.plot(t,y_eu,'-g', linewidth=3.0, label=r'Euler')
plt.legend(loc=4) 
plt.show();

We find that the result matches accurately the exact solution. The value after 25 steps is $y_{\rm{RK4}}(0.5) = 0.255342 $, which is the same as the exact solution (versus an error of order 0.01 in the Euler method). Even if we make the time step 5 times larger, Runge-Kutta 4 preserves its accuracy.

Lastly, there are other higher-order methods for ODEs. A widely used family of methods is the so-called [Adams linear multistep methods](https://en.wikipedia.org/wiki/Linear_multistep_method).  Methods such as Runge–Kutta take some intermediate steps (for example, a half-step) to obtain a higher order method, but then discard all previous information before taking a second step. Multistep methods reuse the information from previous steps rather than discard it in order to improve accuracy. Consequently, multistep methods refer to several previous points and derivative values. In the case of linear multistep methods, a linear combination of the previous points and derivatives values is used. The Adams-Bashfort (explicit) and Adams-Moulton (implicit) methods are the most popular within the linear multistep method family. 

Other methods for ODEs include Extrapolation methods, the Bulirsch-Stoer method, Multivalue methods and Predictor-Corrector methods. You can read about them in Newman's and Landau's books.

## 3. Summary

We have learned how to use Euler's method to solve first-order ODEs numerically. It is the simplest method in this context and it is easy to implement.<br>
We have also learned that:
1. There are more accurate methods for solving ODEs $-$ when possible, use the Runge-Kutta method.
2. There are more stable methods for solving ODEs $-$ when needed (ODE is stiff), use an implicit method.

Euler's method is sometimes useful get an intuition for the solution, 
but Runge-Kutta is recommended as a general method for solving ODEs.<br> 
When ODEs are stiff, one should try using an implicit method.

## References
J. Butcher, Numerical Methods for Ordinary Differential Equations (Wiley, 2nd Ed.)